In [22]:
#  !pip install langchain
# !pip install langchain openai
# !pip install faiss-cpu
# !pip install nltk
!pip install --upgrade openai





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.69.0
    Uninstalling openai-1.69.0:
      Successfully uninstalled openai-1.69.0


In [4]:
# !pip install langchain-community

In [29]:
from langchain.chat_models import ChatOpenAI
import os
import openai
import faiss
import numpy as np
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [13]:
import openai


os.environ["OPENAI_API_KEY"]=grisha_key


In [41]:
llm = ChatOpenAI(temperature=0.1)


In [ ]:
import openai
import numpy as np

import faiss

# Function to get embeddings for a text
def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"  # OpenAI's embedding model
    )
    return np.array(response['data'][0]['embedding'], dtype=np.float32)


# Initialize FAISS index (L2 distance for similarity search)
d = 1536  # Dimension of the embeddings (OpenAI's ada embeddings)
index = faiss.IndexFlatL2(d)  # This is a simple index for demonstration

# Store translations in a dictionary (to fetch translations later)
db = {}

# Function to add text to the FAISS database
def add_to_db(text, translation):
    embedding = get_embedding(text)
    index.add(embedding.reshape(1, -1))  # Add embedding to FAISS index
    db[index.ntotal - 1] = translation  # Save translation against index id


# Function to perform translation
def translate_text(text, target_language):
    # Get embedding for input text
    embedding = get_embedding(text)

    # Search for similar translations in FAISS
    if index.ntotal > 0:
        D, I = index.search(embedding.reshape(1, -1), 1)  # Search for nearest translation
        if D[0][0] < 0.1:  # Threshold for similarity
            return db[I[0][0]]  # If similar, return the translation

    # Otherwise, use GPT-4 for translation
    prompt = f"Translate this text into {target_language}: {text}"

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{
            "role": "system",
            "content": "You are a helpful assistant that provides translations."
        }, {
            "role": "user",
            "content": prompt
        }]
    )
    translation = response['choices'][0]['message']['content'].strip()

    # Store the new translation in FAISS and DB
    index.add(embedding.reshape(1, -1))  # Add to FAISS index
    db[index.ntotal - 1] = translation  # Save translation

    return translation


In [28]:
import openai
import numpy as np

# Function to get embeddings for a text
def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"  # OpenAI's embedding model
    )
    return np.array(response['data'][0]['embedding'], dtype=np.float32)


get_embedding("ich bin viren ")

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [42]:
embedding_model = OpenAIEmbeddings()

# Function to get embedding for a single text
def get_embedding(text):
    # Use embed_documents for embedding the text
    embeddings = embedding_model.embed_documents([text])
    return embeddings[0]  # Return the first embedding


In [43]:
from openai import OpenAI

In [44]:
# Create FAISS Index (flat L2 distance index)
d = 1536  # Dimension of the embedding vectors (text-embedding-ada-002 has 1536 dimensions)
index = faiss.IndexFlatL2(d)

# Store translations in a dictionary (for quick lookup)
db = {}

# Function to add text and its translation to FAISS and db
def add_to_db(text, translation):
    embedding = get_embedding(text)
    index.add(np.array(embedding).reshape(1, -1))  # Add embedding to FAISS
    db[index.ntotal - 1] = translation  # Store translation in dictionary



In [74]:
def translate_text(text, target_language):
    # Get the embedding for the input text
    embedding = get_embedding(text)

    # Search the FAISS index for similar translations
    if index.ntotal > 0:
        D, I = index.search(np.array(embedding).reshape(1, -1), 1)  # Search for nearest match
        if D[0][0] < 0.1:  # Threshold for similarity (you can adjust this)
            return db[I[0][0]]  # Return the translation if a match is found

    # Use GPT-4 to translate the text if no match is found
    prompt = f"Translate the following text to {target_language}: {text}"

    # Generate translation using OpenAI's LLM (via LangChain)
    response = llm.invoke([{
        "role": "system",
        "content": "You are a helpful assistant providing translations."
    }, {
        "role": "user",
        "content": prompt
    }])

    # Access the content of the response (message content)
    translation = response.content.strip()  # Get the content of the first message

    # Store the new translation in the FAISS index and dictionary
    index.add(np.array(embedding).reshape(1, -1))  # Add new embedding to FAISS index
    db[index.ntotal - 1] = translation  # Save translation to the dictionary

    return translation




In [75]:
translate_text("ich bin viren", "english")

'The English translation is: I am viruses.'

In [39]:
# !pip install tiktoken

In [67]:
import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os



# Initialize OpenAI LLM (Large Language Model)
llm = ChatOpenAI(model="gpt-4")

# Function to generate translation
def translate_text(text, target_language):
    # Construct the prompt to translate the text
    prompt = f"Translate the following text to {target_language}: {text}"

    # Generate translation using OpenAI's LLM (via LangChain)
    response = llm.invoke([{
        "role": "system",
        "content": "You are a helpful assistant providing translations."
    }, {
        "role": "user",
        "content": prompt
    }])

    # Extract the translation from the response
    translation = response.content.strip()  # Get the content of the first message

    return translation


In [68]:
translate_text("i am viren vaviya","german")

'Ich bin Viren Vaviya'

In [69]:
target_language = "german"
text = "i am viren"

# Construct the prompt to translate the text
prompt = f"Translate the following text to {target_language}: {text}"

    # Generate translation using OpenAI's LLM (via LangChain)
response = llm.invoke([{
        "role": "system",
        "content": "You are a helpful assistant providing translations."
    }, {
        "role": "user",
        "content": prompt
    }])

translation=response.content.strip()

In [72]:
import re
cleaned_translation = re.sub(r'[^a-zA-Z\s]', '', translation)
# matches = re.findall(r'\b\w+\b', translation)

In [73]:
print("Original Translation:", translation)
print("Cleaned Translation:", cleaned_translation)
print("Matches found with regex:", matches)

Original Translation: Ich bin Viren.
Cleaned Translation: Ich bin Viren
Matches found with regex: ['Ich', 'bin', 'Viren']


In [76]:
import re
from langchain.chat_models import ChatOpenAI

# Initialize the OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to split text into chunks
def split_text(text, max_words=500):
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Split at sentence boundaries
    chunks = []
    current_chunk = []
    word_count = 0

    for sentence in sentences:
        words = sentence.split()
        if word_count + len(words) > max_words:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            word_count = 0
        current_chunk.append(sentence)
        word_count += len(words)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Function to translate text chunk-by-chunk
def translate_large_text(text, target_language):
    chunks = split_text(text)
    translated_chunks = []

    for chunk in chunks:
        prompt = f"Translate the following text to {target_language}: {chunk}"
        response = llm.invoke([{"role": "user", "content": prompt}])
        translated_text = response.content.strip()
        translated_chunks.append(translated_text)

    return " ".join(translated_chunks)

# Example Usage
input_text = "ich bin viren"
target_language = "English"

translated_text = translate_large_text(input_text, target_language)
print("Translated Text:", translated_text)


Translated Text: The translation of "ich bin viren" from German to English is "I am viruses." However, this seems like an unusual statement. If you meant to say "I have viruses," the correct German would be "ich habe Viren."


In [79]:
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text **accurately** into {target_language} while preserving meaning, tone, and style.
    If the text contains idioms, slang, or culturally specific references, translate them to an **equivalent** phrase in {target_language}.
    Ensure proper grammar, sentence structure, and fluency in {target_language}.

    Text:
    "{text}"
    """

# Example usage
text = "It's raining cats and dogs."
target_language = "German"
prompt = get_translation_prompt(text, target_language)
print(prompt)



    You are a professional translator.
    Translate the following text **accurately** into German while preserving meaning, tone, and style.
    If the text contains idioms, slang, or culturally specific references, translate them to an **equivalent** phrase in German.
    Ensure proper grammar, sentence structure, and fluency in German.

    Text:
    "It's raining cats and dogs."
    


In [80]:
from langchain.chat_models import ChatOpenAI

# Initialize the OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to generate translation prompt
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text **accurately** into {target_language} while preserving meaning, tone, and style.
    If the text contains idioms, slang, or culturally specific references, translate them to an **equivalent** phrase in {target_language}.
    Ensure proper grammar, sentence structure, and fluency in {target_language}.

    Text:
    "{text}"
    """

# Function to get translation from OpenAI
def translate_text(text, target_language):
    prompt = get_translation_prompt(text, target_language)

    response = llm.invoke([
        {"role": "user", "content": prompt}
    ])

    return response.content.strip()  # Extracting the response content

# Example usage
text = "It's raining cats and dogs."
target_language = "German"
translation = translate_text(text, target_language)

print("Translation:", translation)


Translation: Es regnet Bindfäden.


In [82]:
from langchain.chat_models import ChatOpenAI

# Initialize the OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to generate translation prompt
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text into {target_language} while preserving its true meaning, tone, and **cultural context**.
    Do not translate idioms, slang, or culturally specific phrases literally. Instead, use an equivalent expression in {target_language}.
    Maintain **formal grammar** and **natural fluency**.

    Text: "{text}"
    """

# Function to get translation from OpenAI
def translate_text(text, target_language):
    prompt = get_translation_prompt(text, target_language)

    response = llm.invoke([
        {"role": "user", "content": prompt}
    ])

    return response.content.strip()  # Extracting the response content

# Example usage
# text = "It's raining cats and dogs."  # English idiom
text="ich bin viren"
target_language = "English"
translation = translate_text(text, target_language)

print("Translation:", translation)


Translation: I am ill.


In [84]:
from langchain.chat_models import ChatOpenAI

# Initialize the OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to generate translation prompt
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text into {target_language} while maintaining **accuracy, fluency, and natural tone**.
    - **Simple words** should be translated **directly** without adding unnecessary complexity.
    - **Idioms, slang, and cultural references** should be translated using an **equivalent expression** in {target_language}, not word-for-word.
    - Maintain **formal grammar** where appropriate.

    Text: "{text}"
    """

# Function to get translation from OpenAI
def translate_text(text, target_language):
    prompt = get_translation_prompt(text, target_language)

    response = llm.invoke([
        {"role": "user", "content": prompt}
    ])

    return response.content.strip()  # Extracting the response content

# Example usage
# text = "Hello, my name is Viren. It's raining cats and dogs."
text="i am viren vaviya"
target_language = "English"
translation = translate_text(text, target_language)

print("Translation:", translation)


Translation: My name is Viren Vaviya.


In [85]:
from langchain.chat_models import ChatOpenAI

# Initialize the OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to generate translation prompt
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text into {target_language} while maintaining **accuracy, fluency, and natural tone**.
    - **Simple words** should be translated **directly** without adding unnecessary complexity.
    - **Idioms, slang, and cultural references** should be translated using an **equivalent expression** in {target_language}, not word-for-word.
    - Maintain **formal grammar** where appropriate.

    Text: "{text}"
    """

# Function to translate text into any language
def translate_text(text, target_language):
    prompt = get_translation_prompt(text, target_language)

    response = llm.invoke([
        {"role": "user", "content": prompt}
    ])

    return response.content.strip()  # Extract only the translated text

# Example usage: Translate to different languages
text = "Hello, my name is Viren. It's raining cats and dogs."

languages = ["French", "Spanish", "Chinese", "Hindi", "Arabic", "Russian", "Japanese", "Italian"]

# for lang in languages:
#     translation = translate_text(text, lang)
#     print(f"\n🔹 Translation in {lang}: {translation}")



🔹 Translation in French: Bonjour, je m'appelle Viren. Il pleut des cordes.

🔹 Translation in Spanish: Hola, mi nombre es Viren. Está lloviendo a cántaros.

🔹 Translation in Chinese: 你好，我叫维伦。现在正下着倾盆大雨。

🔹 Translation in Hindi: नमस्ते, मेरा नाम विरेन है। बहुत तेज़ बारिश हो रही है।

🔹 Translation in Arabic: مرحبًا، اسمي فيرين. إنها تمطر بغزارة.

🔹 Translation in Russian: Привет, меня зовут Вирен. Льет как из ведра.

🔹 Translation in Japanese: こんにちは、私の名前はヴィレンです。土砂降りです。

🔹 Translation in Italian: Ciao, mi chiamo Viren. Sta piovendo a catinelle.


In [86]:
# language_helper.py
from langchain.chat_models import ChatOpenAI
import math

# Initialize the OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to generate translation prompt
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text into {target_language} while maintaining **accuracy, fluency, and natural tone**.
    - **Simple words** should be translated **directly** without adding unnecessary complexity.
    - **Idioms, slang, and cultural references** should be translated using an **equivalent expression** in {target_language}, not word-for-word.
    - Maintain **formal grammar** where appropriate.

    Text: "{text}"
    """

# Function to chunk text if it's too long
def chunk_text(text, max_length=2000):
    # Split text into chunks that fit the model's max token limit
    words = text.split()
    chunks = []
    chunk = []

    for word in words:
        chunk.append(word)
        if len(' '.join(chunk)) > max_length:
            chunks.append(' '.join(chunk[:-1]))  # Add the previous chunk
            chunk = [word]  # Start new chunk

    if chunk:
        chunks.append(' '.join(chunk))  # Add the last chunk

    return chunks

# Function to translate text into any language with chunking for long text
def translate_text(text, target_language):
    prompt = get_translation_prompt(text, target_language)

    # If the text is too long, split it into smaller chunks
    chunks = chunk_text(text)

    translation = ""
    for chunk in chunks:
        # Generate the translation for each chunk
        response = llm.invoke([
            {"role": "user", "content": prompt + "\n" + chunk}
        ])
        translation += response.content.strip() + " "  # Combine translated chunks

    return translation.strip()  # Return full translated text after combining chunks


In [87]:
translate_text("i am viren vaviya","French")

'Je suis Viren Vaviya.'

In [88]:
# language_helper.py
from langchain.chat_models import ChatOpenAI
import tiktoken

# Initialize the OpenAI LLM (assuming GPT-4 or GPT-4 Turbo)
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.2, max_tokens=2000)

# Function to generate translation prompt
def get_translation_prompt(text, target_language):
    return f"""
    You are a professional translator.
    Translate the following text into {target_language} while maintaining **accuracy, fluency, and natural tone**.
    - **Simple words** should be translated **directly** without adding unnecessary complexity.
    - **Idioms, slang, and cultural references** should be translated using an **equivalent expression** in {target_language}, not word-for-word.
    - Maintain **formal grammar** where appropriate.

    Text: "{text}"
    """

# Function to calculate the number of tokens in a text
def count_tokens(text):
    enc = tiktoken.get_encoding("cl100k_base")  # Use the appropriate encoding for the model
    return len(enc.encode(text))

# Function to chunk text if it's too long
def chunk_text(text, max_tokens=2000):
    # Split text into chunks that fit the model's max token limit
    words = text.split()
    chunks = []
    chunk = []
    current_tokens = 0

    for word in words:
        chunk.append(word)
        current_tokens = count_tokens(' '.join(chunk))
        if current_tokens > max_tokens:
            chunks.append(' '.join(chunk[:-1]))  # Add the previous chunk
            chunk = [word]  # Start new chunk
            current_tokens = count_tokens(word)

    if chunk:
        chunks.append(' '.join(chunk))  # Add the last chunk

    return chunks

# Function to translate text into any language with chunking for long text
def translate_text(text, target_language):
    prompt = get_translation_prompt(text, target_language)

    # If the text is too long, split it into smaller chunks
    chunks = chunk_text(text)

    translation = ""
    for chunk in chunks:
        # Generate the translation for each chunk
        try:
            response = llm.invoke([  # Use llm.call() or .invoke() depending on the setup
                {"role": "user", "content": prompt + "\n" + chunk}
            ])
            translation += response.content.strip() + " "  # Combine translated chunks
        except Exception as e:
            print(f"Error translating chunk: {e}")
            return None  # Handle error appropriately

    return translation.strip()  # Return full translated text after combining chunks


In [89]:
translate_text("viren vaviya in germany","German")

'Viren Vaviya in Deutschland'